<i><b>Shane Tendo Quantum submission</b></i>

Teammates:

Jessica Ryuzaki

Aryan

Richard

Objective: to factor increasingly larger semiprime integers using pure quantum algorithms.
Results: Successfully managed to factor 143, but with over 30 minutes of wait time. Can successfully point out non-trivial solutions and recommend changing an unfavourable value of a

Step 1: Installing dependencies

In [1]:
import QuantumRingsLib
from QuantumRingsLib import QuantumRegister, AncillaRegister, ClassicalRegister, QuantumCircuit
from QuantumRingsLib import QuantumRingsProvider
from QuantumRingsLib import job_monitor
from QuantumRingsLib import JobStatus
from matplotlib import pyplot as plt
import numpy as np
import math

provider = QuantumRingsProvider(token ='rings-200.B7zx23W0l0wKojTdUTaDFFgkRHwrUlsO', name='shane.tendo@my.uu.edu')
backend = provider.get_backend("scarlet_quantum_rings")
shots = 2048

provider.active_account()

{'name': 'shane.tendo@my.uu.edu',
 'token': 'rings-200.B7zx23W0l0wKojTdUTaDFFgkRHwrUlsO',
 'max_qubits': '200'}

Step 2. Define the core functions

In [2]:
#GCD finding functions
def gcdpos(a, b, r):
    """Compute the greatest common divisor (GCD) using Euclidean algorithm of a**(r//2) + 1"""
    c = a**(r//2) + 1
    stop = c%b
    if stop == 0:
        print("Trivial solution found. Please try a different a")
    while b:
        c, b = b, c % b
    return c
def gcdneg(a, b, r):
    """Compute the greatest common divisor (GCD) using Euclidean algorithm of a**(r//2) + 1"""
    c = a**(r//2) - 1
    while b:
        c, b = b, c % b
    return c
    
#QC defaults
def set_reg(qc, input, q, n):
    """
    Sets a quantum register with an input value.
    Args:
        qc (QuantumCircuit): The quantum circuit to use
        input (int): The number to be stored in the qubit register
        q (QuantumRegister): The qubit register which is to be programmed with the input number
        n (int): The width of the qubit register to use..
    """
    for i in range (0, n):
        if ((1 << i) & input ):
            qc.x(q[i])
    return

#QFT functions
def iqft_cct(qc, b, n):
    """
    The inverse QFT circuit
    Args:
        qc (QuantumCircuit):The quantum circuit
        b (QuantumRegister):The target register
        n (int):The number of qubits in the registers to use
    Returns:
        None
    """
    for i in range (n):
        for j in range (1, i+1):
            # for inverse transform, we have to use negative angles
            qc.cu1(  -np.pi / 2** ( i -j + 1 ), b[j - 1], b[i])
        # the H transform should be done after the rotations
        qc.h(b[i])
    qc.barrier()
    return

def mult_cct(qc, input1, input2, a, b, s, n):
    """
    a*b multiplier circuit using QFT
    Args:
        qc (QuantumCircuit):The quantum circuit
        input1 (int):The multiplicand
        input2 (int):The multiplier
        s (QuantumRegister):The qubit register holding product a * b
            Note: s has two times the length of a and b
        a (QuantumRegister):The qubit register for multiplicand
        b (QuantumRegister):The qubit register for multiplier
        n (int):The length of the registers
    Returns:
        None.
    """
    set_reg(qc, input1, a, n)
    set_reg(qc, input2, b, n)
    for i in range (0, n):
        c_add_qft(qc, b, s, i,  a[i], n)
    return

def c_add_qft(qc, a, b, offset, control, n):
    """
        The controlled Adder using QFT.
        Args:
            qc (QuantumCircuit):The quantum circuit
            a (QuantumRegister):The source register
            b (QuantumRegister):The target register.
            offset (int):The starting qubit in register b to work from
            control (int):The index number of the control qubit
            n (int):The number of qubits in the registers to use
        Returns: None
    """
    c_qft_cct(qc, b, offset, control, n)
    c_add_cct(qc, a, b, offset, control, n)
    c_iqft_cct(qc, b, offset, control, n)
    return

def c_add_cct(qc, a, b, offset, control, n):
    """
        The controlled adder circuit module.
        Args:
            qc (QuantumCircuit):The quantum circuit
            a (QuantumRegister):The source register
            b (QuantumRegister):The target register. Computed value is stored in this register
            offset (int):The starting qubit in register b to work from
            n (int):The number of qubits in the registers to use
        Returns:
            None
    """
    while (n):
        for i in range(n, 0, -1):
            ccu1(qc, math.pi/2**(n - i), control, a[i-1], b[n-1+offset])
        qc.barrier()
        n -= 1
    return



def c_iqft_cct (qc, b, offset, control, n):
    """
        The controlled Inverse-QFT.
        Args:
            qc (QuantumCircuit):The quantum circuit
            b (QuantumRegister):The target register.
            offset (int):The starting qubit in register b to work from
            control (int):The index number of the control qubit
            n (int):The number of qubits in the registers to use
        Returns:
            None
    """
    for i in range (n):
        for j in range (1, i+1):
            # for inverse transform, we have to use negative angles
            ccu1( qc, -math.pi / 2** ( i -j + 1 ), control, b[j - 1+offset], b[i+offset])
        # the H transform should be done after the rotations
        qc.ch(control, b[i+offset])
    qc.barrier()
    return

def c_qft_cct(qc, b, offset, control, n):
    """
        The controlled QFT.
        Args:
            qc (QuantumCircuit:The quantum circuit
            b (QuantumRegister):The target register.
            offset (int):The starting qubit in register b to work from
            control (int):The index number of the control qubit
            n (int):The number of qubits in the registers to use
        Returns:
            None
    """
    while (n):
        qc.ch(control, b[n + offset - 1])
        for i in range (n-1, 0, -1):
            ccu1(qc, math.pi / 2** (n - i), control, b[i - 1 + offset], b[n + offset - 1])
        n -= 1
    qc.barrier()

    return
def ccu1(qc, theta, q0, q1, q2):
    """
    The controlled Adder using QFT.
    Args:
        qc (QuantumCircuit):The quantum circuit
        theta (float):The rotational angle. See :ref:`QuantumCircuit.u1`
        q0 (int):The first control qubit.
        q1 (int):The second control qubit.
        q2 (int):The target qubit.
    Returns:None
    """
    qc.cu1(theta/2, q1, q2)
    qc.cx(q0, q1)
    qc.cu1(-theta/2, q1, q2)
    qc.cx(q0, q1)
    qc.cu1(theta/2, q0, q2)
    return

#The function that runs the whole circuit by creating 3 registers and calling upon the previously defined functions
def test_mult(value, exponent, modulus):
    numberofqubits = max(exponent, modulus, 4) 
    a = QuantumRegister(numberofqubits , 'a')
    b = QuantumRegister(numberofqubits , 'b')
    s = QuantumRegister(numberofqubits*2 , 's')
    c = ClassicalRegister(numberofqubits*2 , 'c')
    qc = QuantumCircuit(a, b, s, c)
    mult_cct(qc, value, exponent, a, b, s, numberofqubits)
    for i in range (numberofqubits*2):
        qc.measure(s[i],c[i])
    # Execute the circuit
    job = backend.run(qc, shots=shots)
    job_monitor(job)
    result = job.result()
    counts = result.get_counts()

    #This is essentially a way for the computer to tell you it's still going
    if (1 < len(counts)):
        print("Error: More than one result!")
        myproduct = 0
    else:
        scounts = str(counts)
        myproduct = int("0b"+scounts[scounts.index("{")+2:scounts.index(":")-1], 2)
    return myproduct

Step 3. Perform the algorithm

In [7]:
# Now for the values you get to put in!
shots = 1024 #no. of circuit runs
N = 51 #The number being factored
x = 11 #Our coprime number

#Iteration variables
x0 = x
r = 0
print("Starting value: ",x)
while x != x0 or r == 0:
    r = r+1
    res = test_mult(x, x0,int(N/4)) 
    print(res)
    x = res %N
    print(x)
#This clever trick rewrites and repeats the entire circuit to scale up to larger semiprime numbers. 
#In theory, this approach will always work for all semiprimes, but it is limited by compute power and time
print ("Result is: ", r)

# Draw the circuit
#qc.draw('mpl') Very large circuit. Do not recommend drawing

Starting value:  11
Job Running
Job Running
Job Done.
Ending Job Monitor
121
19
Job Queued
Job Running
Job Done.
Ending Job Monitor
209
5
Job Queued
Job Running
Job Done.
Ending Job Monitor
55
4
Job Queued
Job Running
Job Done.
Ending Job Monitor
44
44
Job Queued
Job Running
Job Done.
Ending Job Monitor
484
25
Job Queued
Job Running
Job Done.
Ending Job Monitor
275
20
Job Queued
Job Running
Job Done.
Ending Job Monitor
220
16
Job Queued
Job Running
Job Done.
Ending Job Monitor
176
23
Job Queued
Job Running
Job Done.
Ending Job Monitor
253
49
Job Queued
Job Running
Job Done.
Ending Job Monitor
539
29
Job Queued
Job Running
Job Done.
Ending Job Monitor
319
13
Job Queued
Job Running
Job Done.
Ending Job Monitor
143
41
Job Queued
Job Running
Job Done.
Ending Job Monitor
451
43
Job Queued
Job Running
Job Done.
Ending Job Monitor
473
14
Job Queued
Job Running
Job Done.
Ending Job Monitor
154
1
Job Queued
Job Running
Job Done.
Ending Job Monitor
11
11
Result is:  16


Step 4: Post processing

In [8]:
# visualize
#print (counts)
#plot_histogram(counts)

#Apprehends our two prime factors
p = gcdpos(x0, N, r)
q = gcdneg(x0, N, r)
print ("Period: ", r)
print("The prime factors are:",p,q)
#clean up
del p, q, r, N, x0, x, res

Period:  16
The prime factors are: 17 3
